In [1]:
import pandas as pd
data = pd.read_csv('C:/Users/bam/Downloads/Compressed/data-2024/data.csv')

In [3]:
print(data.columns)


Index(['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId',
       'CurrencyCode', 'CountryCode', 'ProviderId', 'ProductId',
       'ProductCategory', 'ChannelId', 'Amount', 'Value',
       'TransactionStartTime', 'PricingStrategy', 'FraudResult'],
      dtype='object')


In [4]:
# Create aggregate features for each customer
aggregate_features = data.groupby('CustomerId').agg(
    total_transaction_amount=('Amount', 'sum'),
    average_transaction_amount=('Amount', 'mean'),
    transaction_count=('TransactionId', 'count'), 
    std_dev_transaction_amount=('Amount', 'std')  
).reset_index()


In [12]:
# Convert 'TransactionStartTime' to datetime
data['TransactionStartTime'] = pd.to_datetime(data['TransactionStartTime'])

# Extract features
data['transaction_hour'] = data['TransactionStartTime'].dt.hour
data['transaction_day'] = data['TransactionStartTime'].dt.day
data['transaction_month'] = data['TransactionStartTime'].dt.month
data['transaction_year'] = data['TransactionStartTime'].dt.year

In [17]:
print(data.columns.tolist())


['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId', 'Amount', 'Value', 'TransactionStartTime', 'PricingStrategy', 'FraudResult', 'transaction_hour', 'transaction_day', 'transaction_month', 'transaction_year', 'ProviderId_ProviderId_2', 'ProviderId_ProviderId_3', 'ProviderId_ProviderId_4', 'ProviderId_ProviderId_5', 'ProviderId_ProviderId_6', 'ProductId_ProductId_10', 'ProductId_ProductId_11', 'ProductId_ProductId_12', 'ProductId_ProductId_13', 'ProductId_ProductId_14', 'ProductId_ProductId_15', 'ProductId_ProductId_16', 'ProductId_ProductId_19', 'ProductId_ProductId_2', 'ProductId_ProductId_20', 'ProductId_ProductId_21', 'ProductId_ProductId_22', 'ProductId_ProductId_23', 'ProductId_ProductId_24', 'ProductId_ProductId_27', 'ProductId_ProductId_3', 'ProductId_ProductId_4', 'ProductId_ProductId_5', 'ProductId_ProductId_6', 'ProductId_ProductId_7', 'ProductId_ProductId_8', 'ProductId_ProductId_9', 'ProductCategory_data_bundles', 'ProductCategory_financial_services', 

In [19]:
from sklearn.preprocessing import LabelEncoder
# One-Hot Encoding for existing categorical columns
one_hot_columns = ['ChannelId_ChannelId_2', 'ChannelId_ChannelId_3', 'ChannelId_ChannelId_5']

# Check for missing columns and apply One-Hot Encoding
existing_one_hot_columns = [col for col in one_hot_columns if col in data.columns]
if existing_one_hot_columns:
    data = pd.get_dummies(data, columns=existing_one_hot_columns, drop_first=True)
    print(f"One-hot encoded columns: {existing_one_hot_columns}")
else:
    print(f"No columns found for one-hot encoding from: {one_hot_columns}")

# Label Encoding for PricingStrategy and FraudResult
label_encoder = LabelEncoder()
label_encode_columns = ['PricingStrategy', 'FraudResult']

for column in label_encode_columns:
    if column in data.columns:
        data[column] = label_encoder.fit_transform(data[column])
        print(f"Label encoded column: {column}")
    else:
        print(f"Column {column} not found for label encoding.")

# Check the transformed data
print("Transformed DataFrame:")
print(data.head())

One-hot encoded columns: ['ChannelId_ChannelId_2', 'ChannelId_ChannelId_3', 'ChannelId_ChannelId_5']
Label encoded column: PricingStrategy
Label encoded column: FraudResult
Transformed DataFrame:
         TransactionId         BatchId       AccountId       SubscriptionId  \
0  TransactionId_76871   BatchId_36123  AccountId_3957   SubscriptionId_887   
1  TransactionId_73770   BatchId_15642  AccountId_4841  SubscriptionId_3829   
2  TransactionId_26203   BatchId_53941  AccountId_4229   SubscriptionId_222   
3    TransactionId_380  BatchId_102363   AccountId_648  SubscriptionId_2185   
4  TransactionId_28195   BatchId_38780  AccountId_4841  SubscriptionId_3829   

        CustomerId   Amount  Value      TransactionStartTime  PricingStrategy  \
0  CustomerId_4406   1000.0   1000 2018-11-15 02:18:49+00:00                2   
1  CustomerId_4406    -20.0     20 2018-11-15 02:19:08+00:00                2   
2  CustomerId_4683    500.0    500 2018-11-15 02:44:21+00:00                2   
3   C

In [20]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer

In [23]:
# Check for missing values
missing_values = data.isnull().sum()
print("Missing values in each column:")
print(missing_values[missing_values > 0])

# Impute missing values
# Choose imputation strategy
imputation_strategy = 'mean'  

# Identify numerical columns for imputation
numerical_columns = ['Amount', 'Value']  # Numerical columns

# Create imputer for numerical columns
imputer = SimpleImputer(strategy=imputation_strategy)

# Apply imputation
data[numerical_columns] = imputer.fit_transform(data[numerical_columns])
print("Missing values after imputation:")
print(data.isnull().sum())

# Normalize and Standardize Numerical Features
# Normalize selected numerical columns
normalizer = MinMaxScaler()
data[numerical_columns] = normalizer.fit_transform(data[numerical_columns])
print("Data after normalization:")
print(data[numerical_columns].head())

# Standardize selected numerical columns
standardizer = StandardScaler()
data[numerical_columns] = standardizer.fit_transform(data[numerical_columns])
print("Data after standardization:")
print(data[numerical_columns].head())

# Final DataFrame
print("Final DataFrame:")
print(data.head())

# save the processed DataFrame
data.to_csv('processed_bati_bank_data.csv', index=False)

Missing values in each column:
Series([], dtype: int64)
Missing values after imputation:
TransactionId                         0
BatchId                               0
AccountId                             0
SubscriptionId                        0
CustomerId                            0
Amount                                0
Value                                 0
TransactionStartTime                  0
PricingStrategy                       0
FraudResult                           0
transaction_hour                      0
transaction_day                       0
transaction_month                     0
transaction_year                      0
ProviderId_ProviderId_2               0
ProviderId_ProviderId_3               0
ProviderId_ProviderId_4               0
ProviderId_ProviderId_5               0
ProviderId_ProviderId_6               0
ProductId_ProductId_10                0
ProductId_ProductId_11                0
ProductId_ProductId_12                0
ProductId_ProductId_13         

In [26]:
import numpy as np
from datetime import datetime
import pytz 

In [28]:
# Ensure TransactionStartTime is in datetime format
data['TransactionStartTime'] = pd.to_datetime(data['TransactionStartTime'])

# Convert TransactionStartTime to timezone-naive
data['TransactionStartTime'] = data['TransactionStartTime'].dt.tz_localize(None)

# Calculate Recency, Frequency, Monetary Score (RFMS)
now = datetime.now()  

rfms = data.groupby('CustomerId').agg(
    recency=('TransactionStartTime', lambda x: (now - x.max()).days),  
    frequency=('TransactionId', 'count'),  
    monetary=('Amount', 'sum')  
).reset_index()

# Normalize the metrics (optional, for better boundary establishment)
rfms['recency'] = (rfms['recency'] - rfms['recency'].min()) / (rfms['recency'].max() - rfms['recency'].min())
rfms['frequency'] = (rfms['frequency'] - rfms['frequency'].min()) / (rfms['frequency'].max() - rfms['frequency'].min())
rfms['monetary'] = (rfms['monetary'] - rfms['monetary'].min()) / (rfms['monetary'].max() - rfms['monetary'].min())

# Create RFMS score (you can adjust the weights)
rfms['RFMS_score'] = 0.4 * rfms['recency'] + 0.4 * rfms['frequency'] + 0.2 * rfms['monetary']

# View RFMS DataFrame
print(rfms.head())


        CustomerId   recency  frequency  monetary  RFMS_score
0     CustomerId_1  0.922222   0.000000  0.615480    0.491985
1    CustomerId_10  0.922222   0.000000  0.615480    0.491985
2  CustomerId_1001  0.988889   0.000978  0.615495    0.519046
3  CustomerId_1002  0.288889   0.002445  0.615234    0.239580
4  CustomerId_1003  0.133333   0.001222  0.615463    0.176915


In [29]:
# Establish a threshold for classification
threshold = rfms['RFMS_score'].median()  # You can also set a custom threshold

# Classify users
rfms['label'] = np.where(rfms['RFMS_score'] >= threshold, 'good', 'bad')

# View labeled RFMS DataFrame
print(rfms[['CustomerId', 'RFMS_score', 'label']].head())


        CustomerId  RFMS_score label
0     CustomerId_1    0.491985  good
1    CustomerId_10    0.491985  good
2  CustomerId_1001    0.519046  good
3  CustomerId_1002    0.239580  good
4  CustomerId_1003    0.176915   bad


In [31]:

# Sample DataFrame with CustomerId, RFMS_score, and label
data = {
    'CustomerId': ['CustomerId_1', 'CustomerId_10', 'CustomerId_1001', 'CustomerId_1002', 'CustomerId_1003'],
    'RFMS_score': [0.491985, 0.491985, 0.519046, 0.239580, 0.176915],
    'label': ['good', 'good', 'good', 'good', 'bad']
}

rfms = pd.DataFrame(data)

# Function to calculate WoE and IV
def calculate_woe_iv(data, target, feature):
    # Total counts of 'good' and 'bad'
    total_good = data[target].value_counts().get('good', 0)
    total_bad = data[target].value_counts().get('bad', 0)

    # Create a DataFrame for WoE calculation
    woe_df = data[[feature, target]].copy()
    woe_df['good'] = np.where(woe_df[target] == 'good', 1, 0)
    woe_df['bad'] = np.where(woe_df[target] == 'bad', 1, 0)

    # Group by feature and calculate sums
    woe_summary = woe_df.groupby(feature).agg(
        good=('good', 'sum'),
        bad=('bad', 'sum')
    ).reset_index()

    woe_summary['good'] = woe_summary['good'].replace(0, np.nan)
    woe_summary['bad'] = woe_summary['bad'].replace(0, np.nan)

    woe_summary['good_dist'] = woe_summary['good'] / total_good if total_good > 0 else 0
    woe_summary['bad_dist'] = woe_summary['bad'] / total_bad if total_bad > 0 else 0
    
    woe_summary['woe'] = np.log(woe_summary['good_dist'] / woe_summary['bad_dist']).replace({np.inf: np.nan, -np.inf: np.nan}).fillna(0)

    # Calculate IV
    woe_summary['IV'] = (woe_summary['good_dist'] - woe_summary['bad_dist']) * woe_summary['woe']

    iv_value = woe_summary['IV'].sum()
    
    return woe_summary, iv_value

# Perform WoE and IV calculation
woe_result, iv_value = calculate_woe_iv(rfms, 'label', 'RFMS_score')

# Display WoE results
print(woe_result)
print(f"Overall IV value: {iv_value}")


   RFMS_score  good  bad  good_dist  bad_dist  woe  IV
0    0.176915   NaN  1.0        NaN       1.0  0.0 NaN
1    0.239580   1.0  NaN       0.25       NaN  0.0 NaN
2    0.491985   2.0  NaN       0.50       NaN  0.0 NaN
3    0.519046   1.0  NaN       0.25       NaN  0.0 NaN
Overall IV value: 0.0
